In [ ]:
import numpy as np
from scipy import stats
import scipy
from matplotlib import pyplot as plt

In [ ]:

def gene_tolist(file):
    l =[]
    f = open(file, 'r')
    for line in f:
        if 'locus' in line:
            l_header = line.strip().split('\t')
        else:
            s = line.strip().split('\t')
            dic = dict(zip(l_header, s))
            gene = dic['SYMBOL']
            l.append(gene)
    return (l)
            
l_NTD=gene_tolist("NTD_DamMC2_woINVALID.den_input") #DamMC2 without six invalidated calls
print (len(l_NTD))   
l_SSC=gene_tolist("SSC_DamMC2.den_input")
print (len(l_SSC)) 
l_random = []
f_random = open("9606.protein.info.v12.0.txt", 'r') #STRING database
dic_convert = {}
for line in f_random:
    if '#' not in line:
        l_random.append(line.split('\t')[1])
        dic_convert[line.split('\t')[0]] = line.split('\t')[1]
print (len(l_random))
l_random2 = list(set(l_random) - set(l_NTD)-set(l_SSC))
print (len(l_random2)) #ALl genes that doesn't contain NTD and SSC genes


In [ ]:
#extract NTD and SSC interaction from the random set 
f_int_random = open("9606.protein.links.v12.0.txt", 'r')
out_int_NTD_from_random = open("9606.protein.links.v12.0.converted_NTD.txt", 'w')
out_int_SSC_from_random = open("9606.protein.links.v12.0.converted_SSC.txt", 'w')
for line in f_int_random:
    if 'protein' in line:
        out_int_NTD_from_random.write('#' + line )
        out_int_SSC_from_random.write('#' + line )
        pass
    else:
        s = line.split()
        node1 = s[0]
        node2 = s[1]
        s[0]= dic_convert[node1]
        s[1] = dic_convert[node2]
        if s[0] in l_NTD and s[1] in l_NTD:
            #print (s)
            out_int_NTD_from_random.write('\t'.join(s) + '\n')
        if s[0] in l_SSC and s[1] in l_SSC:
            out_int_SSC_from_random.write('\t'.join(s) + '\n')

In [ ]:
#Go for the network colocalization

In [ ]:
import random
size_subset = int(round(len(l_SSC)*0.8,0))
print (size_subset)
l_sub_NTD = []
l_sub_random = []
n_iter = 100000
for i in range(n_iter):
    l_sub_NTD.append(random.sample(l_NTD, k=size_subset))
    #l_sub_SSC.append(random.sample(l_SSC, k=size_subset))
    l_sub_random.append(random.sample(l_random, k=size_subset))


print (len(l_sub_random))

f_int_NTD = open("9606.protein.links.v12.0.converted_NTD.txt", 'r')
dic_edges_NTD ={}
l_cnt_NTD = []
l_cnt_random = []
for line in f_int_NTD:
    if '#' in line:
        l_header = line.strip().split()
        print (l_header)
    else:
        dic=dict(zip(l_header, line.strip().split('\t')))
        node1 = dic['#protein1']
        node2 = dic['protein2']
        if node1 not in dic_edges_NTD:
            dic_edges_NTD[node1] =  [node2]
        else:
            dic_edges_NTD[node1].append(node2)
        

print (len(dic_edges_NTD))

l_edge_weight_ntd_total = []


for subnet in l_sub_NTD:
    dic_edge_weight_ntd = {}
    cnt = 0
    each_gene_subnet = subnet #e
    for gene in each_gene_subnet:
        if gene in dic_edges_NTD:
            l_counter = dic_edges_NTD[gene] #counters
            for counter in l_counter:
                if counter in each_gene_subnet:
                #print (gene, counter)
                    cnt +=1 
                    if gene not in dic_edge_weight_ntd:
                        dic_edge_weight_ntd[gene] = [counter]
                    elif gene in dic_edge_weight_ntd:
                        dic_edge_weight_ntd[gene].append(counter)
                    if counter not in dic_edge_weight_ntd:
                        dic_edge_weight_ntd[counter] = [gene]
                    else:
                        dic_edge_weight_ntd[counter].append(gene)
    
    l_cnt_NTD.append(cnt)
    l_edge_weight_ntd_total.append(dic_edge_weight_ntd)
    



In [ ]:

out = open("per_node_ntd_100k.txt", 'w')
out.write("cohort\tTurn\tedge_per_node\tedge_cnt\n")
dic_per_node = {}
cnt = 0
for ite in l_edge_weight_ntd_total:
    dic_per_node[cnt] = {}
    each_dic = ite
    
    for gene in each_dic:
        nondup = list(set(each_dic[gene]))
        #print (gene,nondup)
        per_node = len(nondup)
        if per_node not in dic_per_node[cnt]:
            dic_per_node[cnt][per_node] = 1
        else:
            dic_per_node[cnt][per_node] +=1
    cnt +=1
        
for turn in dic_per_node:
    #print (dic_per_node[turn])
    cnt_each_turn = 0
    for pernode in dic_per_node[turn]:
        #print ("YES")
        cnt_each_turn += dic_per_node[turn][pernode]
        out.write('MM\t' + str(turn) + '\t' + str(pernode) + '\t' + str(dic_per_node[turn][pernode]) + '\n')
    Noconnection = size_subset -cnt_each_turn
    out.write('MM\t' + str(turn) + '\t' + str(0) + '\t' + str(Noconnection) + '\n')

In [ ]:
#Control

In [ ]:

size_subset = int(round(len(l_SSC)*0.8,0))
l_sub_SSC = []
l_sub_random = []
n_iter = 100000
for i in range(n_iter):
    #l_sub_NTD.append(random.sample(l_NTD, k=size_subset))
    l_sub_SSC.append(random.sample(l_SSC, k=size_subset))
    l_sub_random.append(random.sample(l_random, k=size_subset))


#print (l_sub_random)

f_int_SSC = open("9606.protein.links.v12.0.converted_SSC.txt", 'r')
dic_edges_SSC ={}
l_cnt_SSC = []
l_cnt_random = []
for line in f_int_SSC:
    if 'protein' in line:
        l_header = line.strip().split()
        print (l_header)
    else:
        dic=dict(zip(l_header, line.strip().split('\t')))
        node1 = dic['#protein1']
        node2 = dic['protein2']
        if node1 not in dic_edges_SSC:
            dic_edges_SSC[node1] =  [node2]
        else:
            dic_edges_SSC[node1].append(node2)
        
#print (dic_edges_NTD)
#print (len(dic_edges_SSC))
#cnt0 =0
l_edge_weight_ssc_total = []

for subnet in l_sub_SSC:
    dic_edge_weight_ssc = {}
    cnt = 0
    each_gene_subnet = subnet #e
    for gene in each_gene_subnet:
        if gene in dic_edges_SSC:
            l_counter = dic_edges_SSC[gene] #counters
            for counter in l_counter:
                if counter in each_gene_subnet:
                #print (gene, counter)
                    cnt +=1
                    if gene not in dic_edge_weight_ssc:
                        dic_edge_weight_ssc[gene] = [counter]
                    elif gene in dic_edge_weight_ssc:
                        dic_edge_weight_ssc[gene].append(counter)
                    if counter not in dic_edge_weight_ssc:
                        dic_edge_weight_ssc[counter] = [gene]
                    else:
                        dic_edge_weight_ssc[counter].append(gene)
    l_edge_weight_ssc_total.append(dic_edge_weight_ssc)
    l_cnt_SSC.append(cnt)
    


In [ ]:
#print (l_edge_weight_ssc_total)
dic_per_node = {}
cnt = 0
for ite in l_edge_weight_ssc_total:
    dic_per_node[cnt] = {}
    each_dic = ite
    
    for gene in each_dic:
        nondup = list(set(each_dic[gene]))
       # print (gene,nondup)
        per_node = len(nondup)
        if per_node not in dic_per_node[cnt]:
            dic_per_node[cnt][per_node] = 1
        else:
            dic_per_node[cnt][per_node] +=1
    cnt +=1
        
for turn in dic_per_node:
    #print (dic_per_node[turn])
    cnt_each_turn = 0
    for pernode in dic_per_node[turn]:
        #print ("YES")
        cnt_each_turn += dic_per_node[turn][pernode]
        out.write('Control\t' + str(turn) + '\t' + str(pernode) + '\t' + str(dic_per_node[turn][pernode]) + '\n')
    Noconnection = size_subset -cnt_each_turn
    out.write('Control\t' + str(turn) + '\t' + str(0) + '\t' + str(Noconnection) + '\n')

In [ ]:
out = open("/projects/ps-gleesonlab8/User/hiyoothere/NTD/11.Network_analysis/231125/input/ntd_coloc_100k_iter_0.8contsize.txt", 'w')
out.write("cohort\tconnection\n")

for i in l_cnt_NTD:
    out.write("MM\t" + str(i) + '\n')
for i in l_cnt_SSC:
    out.write("Control\t" + str(i) + '\n')